In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
!pip install openimages numpy pandas scikit-learn opencv-python bing-image-downloader

In [ ]:
import os
import random
import cv2
import numpy as np
# from openimages.download import download_images
from bing_image_downloader import downloader

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

def load_images_from_folder(folder_path, num_images):
    images = []
    random_files = random.sample(os.listdir(folder_path), num_images)
    for filename in random_files:
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
    return images

def test_model(model, image_path):
    img = cv2.imread(image_path)
    if img is not None:
        img = cv2.resize(img, (64, 64))  # Resize the image to match the input size during training
        img = img.reshape(1, -1) / 255.0  # Flatten and normalize the image
        prediction = model.predict(img)
        if prediction[0] == 1:
            return "calisthenics_spot"
        else:
            return "not_calisthenics_spot"
    else:
        return "Invalid image path or format"

# Set the paths for calisthenics_spot and not_calisthenics_spot folders
calisthenics_folder = "/content/drive/Othercomputers/Meu computador/5-periodo/method/calisthenics_spot"
not_calisthenics_folder = "/content/drive/Othercomputers/Meu computador/5-periodo/method/random_images_dataset"

# Load images from the calisthenics_spot folder
num_calisthenics_images = 6552
calisthenics_images = load_images_from_folder(calisthenics_folder, num_calisthenics_images)

# # Download random images from the Open Images V6 dataset for the not_calisthenics_spot class
num_not_calisthenics_images = 6552
# class_label = "not_calisthenics_spot"
# # download_images(limit=num_not_calisthenics_images, annotations=class_label,
# #                 folder=not_calisthenics_folder)
# downloader.download(query='', limit=num_not_calisthenics_images, output_dir=not_calisthenics_folder, adult_filter_off=False, force_replace=False, timeout=60, verbose=True)

# Load images from the not_calisthenics_spot folder
not_calisthenics_images = load_images_from_folder(not_calisthenics_folder, num_not_calisthenics_images)

# Create labels for the data
calisthenics_labels = np.ones(num_calisthenics_images)
not_calisthenics_labels = np.zeros(num_not_calisthenics_images)

# Combine the data and labels for the model
X = np.array(calisthenics_images + not_calisthenics_images)
y = np.concatenate((calisthenics_labels, not_calisthenics_labels))

# Flatten and normalize the images
X = X.reshape(X.shape[0], -1)
X = X.astype('object') / 255.0

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the SVM classifier
svm_classifier = SVC(kernel='linear', C=1.0)
svm_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

<ipython-input-4-884b39536fcd>:58: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  X = np.array(calisthenics_images + not_calisthenics_images)


In [8]:
import os
import random
import cv2
import numpy as np
# from openimages.download import download_images
from bing_image_downloader import downloader

from sklearn.model_selection import train_test_split
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score

In [9]:
def load_images_from_folder(folder_path, num_images):
    images = []
    random_files = random.sample(os.listdir(folder_path), num_images)
    for filename in random_files:
        img_path = os.path.join(folder_path, filename)
        img = cv2.imread(img_path)
        if img is not None:
            images.append(img)
    return images

def test_model(model, image_path):
    img = cv2.imread(image_path)
    if img is not None:
        img = cv2.resize(img, (64, 64))  # Resize the image to match the input size during training
        img = img.reshape(1, -1) / 255.0  # Flatten and normalize the image
        prediction = model.predict(img)
        if prediction[0] == 1:
            return "calisthenics_spot"
        else:
            return "not_calisthenics_spot"
    else:
        return "Invalid image path or format"


def resize_images_by_percentage(image_folder, percentage, output_folder):
    os.makedirs(output_folder, exist_ok=True)
    resized_images = []

    for filename in os.listdir(image_folder):
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            new_width = int(img.shape[1] - (img.shape[1] * percentage))
            new_height = int(img.shape[0] - (img.shape[0] * percentage))
            resized_img = cv2.resize(img, (new_width, new_height))
            resized_images.append(resized_img)
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, resized_img)

    return np.array(resized_images)

## Resized calisthenics spots

In [12]:
# Set the paths for calisthenics_spot and not_calisthenics_spot folders
calisthenics_folder = "/content/drive/Othercomputers/Meu computador/5-periodo/method/calisthenics_spot"
not_calisthenics_folder = "/content/drive/Othercomputers/Meu computador/5-periodo/method/random_images_dataset"

# Load images from the calisthenics_spot folder
num_calisthenics_images = 6552
calisthenics_images = load_images_from_folder(calisthenics_folder, num_calisthenics_images)

# Specify the percentage by which to resize the images
resize_percentage = 0.8095238

# Resize calisthenics_spot images by the specified percentage
resized_calisthenics_folder = "/content/drive/Othercomputers/Meu computador/5-periodo/method/resized_calisthenics_spotsss"
resized_calisthenics_images = resize_images_by_percentage(calisthenics_folder, resize_percentage, resized_calisthenics_folder)

# # Download random images from the Open Images V6 dataset for the not_calisthenics_spot class
num_not_calisthenics_images = 6552
# class_label = "not_calisthenics_spot"
# # download_images(limit=num_not_calisthenics_images, annotations=class_label,
# #                 folder=not_calisthenics_folder)
# downloader.download(query='', limit=num_not_calisthenics_images, output_dir=not_calisthenics_folder, adult_filter_off=False, force_replace=False, timeout=60, verbose=True)

# Load images from the not_calisthenics_spot folder
not_calisthenics_images = load_images_from_folder(not_calisthenics_folder, num_not_calisthenics_images)

<ipython-input-9-eb366fed2886>:40: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  return np.array(resized_images)


## Already resized

In [13]:
# Set the paths for calisthenics_spot and not_calisthenics_spot folders
calisthenics_folder = "/content/drive/Othercomputers/Meu computador/5-periodo/method/resized_calisthenics_spotsss"
not_calisthenics_folder = "/content/drive/Othercomputers/Meu computador/5-periodo/method/random_images_dataset"

# Load images from the calisthenics_spot folder
num_calisthenics_images = 6552
calisthenics_images = load_images_from_folder(calisthenics_folder, num_calisthenics_images)

# Load images from the not calisthenics_spot folder
num_not_calisthenics_images = 6552
not_calisthenics_images = load_images_from_folder(not_calisthenics_folder, num_not_calisthenics_images)

In [ ]:
a = []
while(1): a.append('1')

In [19]:
# Create labels for the data
calisthenics_labels = np.ones(shape=num_calisthenics_images, dtype=int)
not_calisthenics_labels = np.zeros(shape=num_not_calisthenics_images, dtype=int)

# Combine the data and labels for the model
X = np.array(object=(calisthenics_images + not_calisthenics_images), dtype=object)
y = np.concatenate((calisthenics_labels, not_calisthenics_labels))

# Flatten and normalize the images
X = X.reshape(X.shape[0], -1)
X = X.astype('float32') / 255.0

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create and train the SVM classifier
svm_classifier = SVC(kernel='linear', C=1.0)
svm_classifier.fit(X_train, y_train)

# Make predictions on the test set
y_pred = svm_classifier.predict(X_test)

# Calculate accuracy
accuracy = accuracy_score(y_test, y_pred)
print("Accuracy:", accuracy)

ValueError: ignored

In [ ]:
import os
import cv2
import numpy as np

def calculate_average_image_size(image_folder):
    total_width = 0
    total_height = 0
    num_images = 0

    for filename in os.listdir(image_folder):
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            total_width += img.shape[1]
            total_height += img.shape[0]
            num_images += 1

    average_width = total_width // num_images
    average_height = total_height // num_images

    return average_width, average_height


In [ ]:
def resize_images(image_folder, average_width, average_height, output_folder):
    os.makedirs(output_folder, exist_ok=True)

    for filename in os.listdir(image_folder):
        img_path = os.path.join(image_folder, filename)
        img = cv2.imread(img_path)
        if img is not None:
            width = img.shape[1]
            height = img.shape[0]

            target_width = width * (average_width / width)
            target_height = height * (average_height / height)

            resized_img = cv2.resize(img, (target_width, target_height))
            output_path = os.path.join(output_folder, filename)
            cv2.imwrite(output_path, resized_img)

# Set the paths for calisthenics_spot and not_calisthenics_spot folders
calisthenics_folder = "./spots/calisthenics_spot"
not_calisthenics_folder = "./not_calisthenics_spot"

# Calculate the average size of not_calisthenics_spot images
average_width, average_height = calculate_average_image_size(not_calisthenics_folder)

# Resize calisthenics_spot images to match the average size
resized_calisthenics_folder = "./resized_calisthenics_spot"
resize_images(calisthenics_folder, average_width, average_height, resized_calisthenics_folder)